# Excercise: Frozen Lake


**Mô tả:** Game hồ băng, chia làm 16 ô vuông, mỗi ô vuông sẽ là có băng hoặc không băng. Nếu đi lọt vào ô không băng thì game over! Nhiệm vụ của bạn là di chuyển từ S đến G.

![frozen-lake-description](https://i.imgur.com/jLhoSev.png)
![frozen-lake-map](https://i.imgur.com/1WS7wFJ.png)


**Q-values table**

|State(cell)|Left|Down|Right|Up|
|---|---|---|---|---|
|00 | 0 | 0 | 0 | 0 |
|01 | 0 | 0 | 0 | 0 |
|...| 0 | 0 | 0 | 0 |
|16 | 0 | 0 | 0 | 0 |


In [ ]:
import numpy as np
np.random.seed(1612)
import pandas as pd

## Deterministic Environment

Helper function

In [ ]:
def shortest_path(start,end, Q, actions,max_col):
     path = [start]
     path_actions = []
     cur = start
     while cur != end:
          act = np.argmax(Q[cur])
          path_actions.append(actions[act])
          # turn left so the index -1
          if act == 0:        
               cur -=  1
          # turn down so the index +max_col
          elif act == 1:        
               cur +=  max_col
          # turn right so the index +1
          elif act == 2:         
               cur +=  1
          #turn up so the index -max_col
          else:                    
               cur -= max_col
          path.append(cur)
     return path, path_actions

In [ ]:
def get_next_state(cur_state, act, max_col):
  # TODO check condition at the edges
  # TODO turn left: index - 1: 0
  # TODO turn down: index +  max_col: 1
  # TODO turn right: index + 1: 2
  # TODO turn up: index - max_col: 3
    if cur_state % max_col == 0 and act == 0:
        next_state = cur_state
    elif cur_state % max_col == max_col - 1 and act == 2:
        next_state = cur_state
    elif cur_state < max_col and act == 3:
        next_state = cur_state
    elif cur_state > max_col*max_col-1-max_col and act == 1:
        next_state = cur_state
    else:
        if act == 0:
            next_state = cur_state - 1
        elif act == 1:
            next_state = cur_state + max_col
        elif act == 2:
            next_state = cur_state + 1
        elif act == 3:
            next_state = cur_state - max_col
      
    return next_state

In [ ]:
def planning(iter, Q, gamma, terminal): # đi kiếm q* 
  for i in range(iter): # iter số vòng lặp để mỗi ô đều có reward
    for state in range(Q.shape[0]): # Q.shape = (16, 4) --> 16 states
      if state in terminal:
        continue
      for act in range(Q.shape[1]): # --> 4 actions --> or range(iter)
        reward = 0
        next_state = get_next_state(state, act, max_col=4) # max_col = 4
        if next_state == 15:
          reward = 1
        Q[state, act] = reward + gamma * np.max(Q[next_state]) # gamma = bn tuỳ mình, max(Q[next_state]) ko có act, nghĩa là 4 LRUD cái nào cho Q lớn nhất
  return Q

In [ ]:
# init Q matrix (applied Bellman) to calculate policy
Q = np.zeros((16,4))
# init list of actions
actions = ["left", "down", "right", "up"] 
# init terminal state (holes and goal)
terminal = [5, 7, 11, 12, 15] # terminal = holes
# call the planning() function
Q = planning(1000, Q, 0.8, terminal)
print(Q)
print(Q.shape)
# visualise Q table
# Q[0,2] += 0,00001 # nếu muốn bắt đầu từ right thay vì down 
dict_Q = {actions[0]: Q.T[0], actions[1]: Q.T[1], actions[2]: Q.T[2], actions[3]: Q.T[3]}  # visualize Q to easily understand
print(pd.DataFrame(dict_Q),"\n")
# Find the shortest path to get the target
path, path_actions = shortest_path(0,15, Q, actions, max_col=4)
print("The path to go to the end:", path)
print("The way to go to the end:", path_actions)

[[0.262144 0.32768  0.32768  0.262144]
 [0.262144 0.       0.4096   0.32768 ]
 [0.32768  0.512    0.32768  0.4096  ]
 [0.4096   0.       0.32768  0.32768 ]
 [0.32768  0.4096   0.       0.262144]
 [0.       0.       0.       0.      ]
 [0.       0.64     0.       0.4096  ]
 [0.       0.       0.       0.      ]
 [0.4096   0.       0.512    0.32768 ]
 [0.4096   0.64     0.64     0.      ]
 [0.512    0.8      0.       0.512   ]
 [0.       0.       0.       0.      ]
 [0.       0.       0.       0.      ]
 [0.       0.64     0.8      0.512   ]
 [0.64     0.8      1.       0.64    ]
 [0.       0.       0.       0.      ]]
(16, 4)
        left     down    right        up
0   0.262144  0.32768  0.32768  0.262144
1   0.262144  0.00000  0.40960  0.327680
2   0.327680  0.51200  0.32768  0.409600
3   0.409600  0.00000  0.32768  0.327680
4   0.327680  0.40960  0.00000  0.262144
5   0.000000  0.00000  0.00000  0.000000
6   0.000000  0.64000  0.00000  0.409600
7   0.000000  0.00000  0.00000  0.00000